In [104]:
import os
import pdb
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [105]:
    download_folder = 'C:\\Users\\Angel\\Desktop\\files'

    profile = webdriver.FirefoxProfile()
    profile.set_preference("browser.download.folderList", 2)
    profile.set_preference("browser.download.dir", download_folder)
    profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/octet-stream")


    driver = webdriver.Firefox(firefox_profile=profile)

In [106]:
driver.get('https://www.cenace.gob.mx/Paginas/SIM/Reportes/H_PreciosEnergiaSisMEM.aspx?N=5&opc=divCssPreEnergia&site=Precios%20de%20la%20energ%C3%ADa/Precios%20Marginales%20Locales/MDA/Diarios&tipoArch=C&tipoUni=SIN&tipo=Diarios&nombrenodop=Precios%20Marginales%20Locales')
# https://www.cenace.gob.mx/Paginas/SIM/Reportes/H_PreciosEnergiaSisMEM.aspx?N=6&opc=divCssPreEnergia&site=Precios%20de%20la%20energ%C3%ADa/Precios%20Marginales%20Locales/MDA/Mensuales&tipoArch=C&tipoUni=SIN&tipo=Mensuales&nombrenodop=Precios%20Marginales%20Locales
driver.find_element_by_xpath('/html/body/form/div[4]/div[1]/div/div[1]/div[2]/div/table/tbody/tr/td[1]/div/ul/li[1]/div/span[3]').click()

In [107]:
xpath = '/html/body/form/div[4]/div[1]/div/div[1]/div[2]/div/table/tbody/tr/td[3]/div[1]/div/table/tbody/tr[2]/td[2]/table/tbody/tr[{}]/td[2]/a[1]/img'


In [108]:
# main_handle = driver.current_window_handle
# print(main_handle)


In [109]:
def wait_download(directorio,i):
    
    while directorio == os.listdir(download_folder):
        pass

    time.sleep(1)
    print(f'Descargando {i}', end = '')

    wait = True
    while wait:
        wait = False
        for file in os.listdir(download_folder):
            if ".part" in file:
                time.sleep(0.5)
                wait = True
                print('.', end = '')
    print('Listo')


In [110]:
print('Inicio')
for i in range(1,32):
    
    file_selector = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, xpath.format(i))))

    directorio = os.listdir(download_folder)

    file_selector.click()

    wait_download(directorio,i)

print(f'Listos {i}')

driver.quit()

Inicio
Descargando 1...Listo
Descargando 2.Listo
Descargando 3.Listo
Descargando 4..Listo
Descargando 5..Listo
Descargando 6....Listo
Descargando 7...Listo
Descargando 8.Listo
Descargando 9...Listo
Descargando 10.......Listo
Descargando 11.............Listo
Descargando 12....Listo
Descargando 13....Listo
Descargando 14...Listo
Descargando 15..Listo
Descargando 16..Listo
Descargando 17..Listo
Descargando 18..Listo
Descargando 19....Listo
Descargando 20.....Listo
Descargando 21....Listo
Descargando 22....Listo
Descargando 23....Listo
Descargando 24....Listo
Descargando 25..Listo
Descargando 26..Listo
Descargando 27..Listo
Descargando 28..Listo
Descargando 29..Listo
Descargando 30...Listo
Descargando 31...Listo
Listos 31


In [38]:
import os
import pdb
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# print([os.getcwd()])
download_folder = 'C:\\Users\\Angel\\Desktop\\files'
main_url = 'https://www.cenace.gob.mx/Paginas/SIM/Reportes/H_PreciosEnergiaSisMEM.aspx?N=5&opc=divCssPreEnergia&site=Precios%20de%20la%20energ%C3%ADa/Precios%20Marginales%20Locales/MDA/Diarios&tipoArch=C&tipoUni=SIN&tipo=Mensuales&nombrenodop=Precios%20Marginales%20Locales'
xpath_timelapse = '/html/body/form/div[4]/div[1]/div/div[1]/div[2]/div/table/tbody/tr/td[1]/div/ul/li[2]/div/span[3]'
element_dict = {'file':'/td[2]/a[1]/img', 'month':'/td[1]','date':'/td[3]'}
xpath_files = '/html/body/form/div[4]/div[1]/div/div[1]/div[2]/div/table/tbody/tr/td[3]/div[1]/div/table/tbody/tr[{t}]/td[2]/table/tbody/tr[{r}]{type}'

def wait_download(directorio,i):

    while directorio == os.listdir(download_folder):
        pass

    time.sleep(1)
    print(f'Descargando {i}', end = '')

    wait = True
    while wait:
        wait = False
        for file in os.listdir(download_folder):
            if ".part" in file:
                time.sleep(0.5)
                wait = True
                print('.', end = '')
    print('Listo')

In [41]:
profile = webdriver.FirefoxProfile()
profile.set_preference("browser.download.folderList", 2)
profile.set_preference("browser.download.dir", download_folder)
profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/octet-stream")
print('Opening Browser.')
driver = webdriver.Firefox(firefox_profile=profile)

driver.get(main_url)

driver.find_element_by_xpath(xpath_timelapse).click()

print('Inicio')

for table in range(2,73,2):
    for row in range(1,3):

        xpath = xpath_files.format(t=table, r=row, type=element_dict['file'])

        file_selector = WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.XPATH, xpath)))
        
        directorio = os.listdir(download_folder)

        file_selector.click()
        
        wait_download(directorio,i)

print(f'Listos {i}')

driver.quit()

Opening Browser.
Inicio


ElementClickInterceptedException: Message: Element <img src="../img/bot csv.png"> is not clickable at point (586,345) because another element <div id="ctl00_ContentPlaceHolder1_NotifAvisos_simpleContentDiv" class="rnContent"> obscures it
